In [10]:
import numpy as np
import pandas as pd
import shapely

from src import h3_utils
import utils as U

In [2]:
df = pd.read_csv("/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bng_malls_ranked.csv")

In [4]:
df['competitor_type'] = 'primary_competitor'

In [6]:
df

,id,name,lat,lng,cid,number_of_votes,ratings,created_at,date,number_of_reviews,reviews_per_day,wkt,competitor_type
0,ChIJZ0FegEs9rjsRZFR-C-hVwqQ,Orion Mall,13.011023,77.554715,0x3bae3d4b805e4167:0xa4c255e80b7e5464,183294,4.5,1706081213,1.674391e+09,183294.0,499.728107,POLYGON ((77.55455350014815 13.012127190197445...,primary_competitor
1,ChIJuzGXhU0UrjsRCRsq7sQj5Xs,Nexus Mall Koramangala,12.934777,77.611778,0x3bae144d859731bb:0x7be523c4ee2a1b09,131493,4.4,1706081213,1.660855e+09,131494.0,251.202709,POLYGON ((77.61171273612247 12.935463783047162...,primary_competitor
2,ChIJoRKfACMWrjsRrWqiP9PBT5Y,Mantri Square Mall,12.991630,77.571172,0x3bae1623009f12a1:0x964fc1d33fa26aad,144046,4.4,1706081213,1.653817e+09,144047.0,238.129740,"POLYGON ((77.56961331170093 12.99219738364626,...",primary_competitor
3,ChIJcToFoO4RrjsRiNp1K-jl9oE,Nexus Shantiniketan Mall,12.989536,77.728101,0x3bae11eea0053a71:0x81f6e5e82b75da88,68276,4.5,1706081213,1.671622e+09,68277.0,171.190504,POLYGON ((77.72763860657926 12.990380990107184...,primary_competitor
4,ChIJlb8QkIwXrjsRGUxSwTzIMhQ,Lulu Mall,12.981820,77.563117,0x3bae178c9010bf95:0x1432c83cc1524c19,39828,4.4,1706081213,1.680949e+09,39828.0,136.919544,POLYGON ((77.56289552267329 12.982497418607135...,primary_competitor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,ChIJLfcwwb89rjsRuEH5q3deUfQ,Platinum Mall,12.988786,77.537866,0x3bae3dbfc130f72d:0xf4515e77abf941b8,43,4.3,1706081213,1.462089e+09,43.0,0.015227,"POLYGON ((77.53784923874949 12.98897115145576,...",primary_competitor
67,ChIJoWqmRNwSrjsRESUY1zbg9wo,Decathlon,12.900932,77.707246,0x3bae12dc44a66aa1:0xaf7e036d7182511,0,0.0,1706081213,NaN,NaN,NaN,"POLYGON ((77.7069074202376 12.901542373078865,...",primary_competitor
68,ChIJ6wwILYEWrjsR0CRciJIPFy8,Bangalore Central Mall,12.973381,77.611278,0x3bae16812d080ceb:0x2f170f92885c24d0,0,0.0,1706081213,NaN,NaN,NaN,POLYGON ((77.61142393163342 12.973896482295942...,primary_competitor
69,ChIJez6EsCc9rjsR1OgmK7Vjxvs,Rock Line Mall,13.040773,77.518319,0x3bae3d27b0843e7b:0xfbc663b52b26e8d4,0,0.0,1706081213,NaN,NaN,NaN,POLYGON ((77.51818646390602 13.041153773888459...,primary_competitor


In [5]:
df.to_csv("/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/mall_competitors.csv", index=False)

In [7]:
import geopandas as gpd


def get_competitors(site_id):
    def get_anchor_brands(site_id):
        g = U.get_project_info(id=site_id).query("catchment_type=='i1000mtb'")['geometry'].tolist()[0]
        g = shapely.from_wkt(g)
        stores_gdf = gpd.GeoDataFrame([{"store_name": 1, "geometry": g}], geometry='geometry')
        coffee_shops = bng_data.query("(category=='coffee_shop') and (brand_id!='N_A')")
        coffee_shops_gdf = gpd.GeoDataFrame(coffee_shops,
                                            geometry=gpd.points_from_xy(coffee_shops['lng'], coffee_shops['lat']))
        coffee_shops_count = coffee_shops.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index()
        cat_anchor = stores_gdf.sjoin(coffee_shops_gdf, how='inner')[coffee_shops.columns.tolist()]
        cat_anchor = cat_anchor.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index()
        cat_anchor = cat_anchor.merge(coffee_shops_count, how='inner', on='brand_id')
        cat_anchor['rank'] = cat_anchor['count_y'] / cat_anchor['count_x']
        cat_anchor = cat_anchor.sort_values(by='rank', ascending=False)['brand_id'].values.tolist()[:5]
        return cat_anchor

    bng_data = pd.read_csv(
        '/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv')

    primary = bng_data.query("brand_id!='N_A' and category=='gym_fitness'")
    primary_brands = primary.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index().query(
        "count>5").sort_values(by='count', ascending=False)['brand_id'].tolist()
    df = pd.read_csv(
        f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{site_id}/raw/location_poi_data.csv")
    mask1 = df['brand_id'].isin(primary_brands)
    df.loc[mask1, 'competitor_type'] = 'primary_competitor'
    mask2 = (df['category'] == 'gym_fitness') & (~df['brand_id'].isin(primary_brands)) & (df['number_of_votes'] > 50)
    df.loc[mask2, 'competitor_type'] = 'secondary_competitor'
    mask3 = df['brand_id'].isin(get_anchor_brands(site_id))
    df.loc[mask3, 'competitor_type'] = 'anchor_competitor'
    df = df.query("~competitor_type.isna()")
    return df





In [7]:
def get_competitors(site_id):
    primary_brands = ['snapfitnessindia',
                      'stayfitindia',
                      'vivtaa',
                      'planetfitness',
                      'goldsgym',
                      'nammacrossfit']
    anchor_brands = [

        "zomato_domino's_pizza",
        'ratnadeep',
        'ril',
        'dominos',
        'dmartindia',
        'naturesbasket',
        'marksandspencer',
        'relianceretail',
        'dmartreadyonline',
        'naturals',
        'zudio',
        'lenskart',
        'reliancesmartbazaar',
        'vishalmegamart',
        'reliancesmartpoint',
        'toniandguy',
        'dmart']
    df = pd.read_csv(
        f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{site_id}/raw/location_poi_data.csv")
    mask1 = df['brand_id'].isin(primary_brands)
    df.loc[mask1, 'competitor_type'] = 'primary_competitor'
    mask2 = (df['category'] == 'gym_fitness') & (~df['brand_id'].isin(primary_brands)) & (df['number_of_votes'] > 20)
    df.loc[mask2, 'competitor_type'] = 'secondary_competitor'
    mask3 = df['brand_id'].isin(anchor_brands)
    df.loc[mask3, 'competitor_type'] = 'anchor_competitor'
    df = df.query("~competitor_type.isna()")
    return df

In [9]:
sids = ['130631_776205', '128873_775969']
for sid in sids[:]:
    df = get_competitors(sid)
    df.to_csv(f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{sid}/competitors.csv")

In [20]:
df['competitor_type'].value_counts()

competitor_type
secondary_competitor    141
anchor_competitor        53
Name: count, dtype: int64

In [69]:
df.to_csv(f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{sid}/competitors.csv")

In [13]:
bng_data = pd.read_csv(
    '/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv')

In [14]:
primary = bng_data.query("brand_id!='N_A' and category=='gym_fitness'")
primary_brands = primary.groupby('brand_id').agg(count=("id", np.count_nonzero)).reset_index().query(
    "count>5").sort_values(by='count', ascending=False)


In [21]:
secondary_brands = bng_data.query(
    "category=='gym_fitness' and number_of_votes>50 and brand_id not in(@primary_brands['brand_id']) ")
secondary_brands

,id,name,lat,lng,cid,brand_id,brand_name,number_of_votes,place_url,source,category,top_category,type,created_at
6,ChIJawgheE0TrjsRjbLaEk_6Euo,Barbell Cartel Gym,12.957161,77.736313,0x3bae134d7821086b:0xea12fa4f12dab28d,N_A,N_A,167,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
231,ChIJkdKgmYA9rjsRY-ml6pDMNAI,FEMININE WOMEN'S ONLY GYM Nagarabhavi,12.960160,77.506882,0x3bae3d8099a0d291:0x234cc90eaa5e963,N_A,N_A,62,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
344,ChIJubahgNoXrjsRT5pniThZZjA,SUPRAD Badminton Academy,13.050083,77.607834,0x3bae17da80a1b6b9:0x3066593889679a4f,N_A,N_A,192,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
485,ChIJlbPzOWkRrjsRCol-7vLx-wI,DRDO Phase II Sports Complex,12.986635,77.681442,0x3bae116939f3b395:0x2fbf1f2ee7e890a,N_A,N_A,391,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
593,ChIJs4uw8y0TrjsRlEGp806yPz0,The Professional Fitness Center (Team:TPF) ದಿ ...,12.904058,77.676003,0x3bae132df3b08bb3:0x3d3fb24ef3a94194,N_A,N_A,264,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362156,ChIJDZQzV50VrjsRfh_J06vTLLk,LL PG for MEN,12.953691,77.578125,0x3bae159d5733940d:0xb92cd3abd3c91f7e,N_A,N_A,53,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
362165,ChIJL2Sp-TAVrjsRMl4htzgNf8M,GOLDEN FIZZIK GYM,12.928550,77.596794,0x3bae1530f9a9642f:0xc37f0d38b7215e32,N_A,N_A,209,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
362306,ChIJ965kur8NrjsRDfcAzoSTxxY,GET FIT WITH SHEHI (Personal Trainer),12.989023,77.754524,0x3bae0dbfba64aef7:0x16c79384ce00f70d,N_A,N_A,118,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039
362323,ChIJn5eOvwoXrjsR7U0C4mhV0Kg,Refuel fitness studio,13.014141,77.639542,0x3bae170abf8e979f:0xa8d05568e2024ded,N_A,N_A,63,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,gym_fitness,leisure,N_A,1687847039


In [23]:
anchor = bng_data.query("(category=='coffee_shop') and (brand_id!='N_A')").groupby('brand_id').agg(
    count=("id", np.count_nonzero)).reset_index()
anchor

,brand_id,count
0,1by2foods,1
1,3mangos,3
2,amrutchaha,1
3,artscafe,3
4,avoke,3
...,...,...
120,tribalbrew,3
121,vaamanamruttulya,1
122,vidyacoffee,7
123,wafl,1


In [26]:
site_id = '129065_775925'
df = pd.read_csv(
    f"/home/jyotiraditya/PycharmProjects/SiteReports/data/projects_data/{site_id}/raw/location_poi_data.csv")

In [30]:
cat_anchor = df.query("(category=='coffee_shop') and (brand_id!='N_A')").groupby('brand_id').agg(
    count=("id", np.count_nonzero)).reset_index()
cat_anchor = cat_anchor.merge(anchor, how='inner', on='brand_id')
cat_anchor['rank'] = cat_anchor['count_y'] / cat_anchor['count_x']
cat_anchor = cat_anchor.sort_values(by='rank', ascending=False)
cat_anchor


,brand_id,count_x,count_y,rank
18,hattikaapi,2,29,14.500000
36,slay,1,12,12.000000
24,lassishop,3,31,10.333333
37,smoor,1,10,10.000000
1,bluetokaicoffee,1,10,10.000000
6,chaayos,3,29,9.666667
22,lassicorner,4,38,9.500000
3,cafecoffeeday,14,129,9.214286
10,chaipoint,10,83,8.300000
42,teabarind,1,8,8.000000
